In [1]:
import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(seed=315)

In [2]:
nside = 768
Np_og = 1350
binsize=1
Np    = int(Np_og//binsize)
print(Np)
nskew = nside**2
pix_spacing = 67.5/Np
print(pix_spacing)

1350
0.05


In [3]:
# match the total number of pixels masked to what is necessary for the skyline version
skyline_width = 30
Npmasked_total = skyline_width * nskew
Npm_per_skew = skyline_width
masked_frac = Npmasked_total/(Np*nskew)
print("Percentage masked:", masked_frac*100)

Percentage masked: 2.2222222222222223


In [10]:
# in the subselected forest, do some masking schemes along the line-of-sight axis
# 1: random masking of 1-5 pixels in each skewer for every line of sight
print("Doing random masking")

random_mask = np.ones((nside, nside, Np)) # store the mask

masklocs_per_skewer = []
nmasked_rand = 0
for i in range(nside):
    for j in range(nside):
        nmask = rng.integers(Npm_per_skew-int(skyline_width/2.), Npm_per_skew+int(skyline_width/2.)+1) # average # of pixels masked will be Npm_per_skew
        if nmask != 0:
            masklocs = rng.choice(np.arange(Np), size=nmask, replace=False)
            masklocs_per_skewer.append(masklocs)
            nmasked_rand += nmask
            random_mask[i,j, masklocs] = 0


print("Total number of pixels masked in random scheme is", nmasked_rand/Npmasked_total*100, "percent of the total desired")

# 2: 'skyline-like' masking: mask a range of N pixels 1/x of the way through the box
print("Doing single-skyline masking")
boxdiv = 4
sky_maskrange = np.arange(Np//boxdiv, Np//boxdiv+skyline_width)
nmasked_skyline = skyline_width*nskew
print("total number of pixels masked in skyline scheme is", nmasked_skyline/Npmasked_total*100, "percent of the total desired")
skyline_mask = np.ones((nside, nside, Np)) # store the mask
skyline_mask[:, :, sky_maskrange] = 0

# 3: put a DLA-like mask (400 pixels at once) at random positions
print("Doing DLA masking")
dla_width = 400
print("DLA width in Mpc is", dla_width*pix_spacing)
ndla = int(Npmasked_total // dla_width)
nmasked_dla = 0
dla_mask = np.ones((nside, nside, Np)) # store the mask
# get all indices
dla_locs_xx, dla_locs_yy = np.indices((nside, nside))
if ndla!=nside**2:
    dla_locs_xx = rng.choice(dla_locs_xx.ravel(), size=ndla, replace=False)
    dla_locs_yy = rng.choice(dla_locs_yy.ravel(), size=ndla, replace=False)
else:
    dla_locs_xx = dla_locs_xx.ravel()
    dla_locs_yy = dla_locs_yy.ravel()
dla_locs_zstart = rng.choice(np.arange(Np-dla_width), size=ndla, replace=True)
for i in range(ndla):
    z_start = dla_locs_zstart[i]
    z_end  = z_start + dla_width
    nmasked_dla += dla_width
    dla_mask[dla_locs_xx[i], dla_locs_yy[i], z_start:z_end] = 0
print("total number of pixels masked in DLA scheme is", nmasked_dla/Npmasked_total*100, "percent of the total desired")


# 4: put another DLA-like mask (40 pixels at once) at random positions
print("Doing small DLA masking")
dla_width = 40
print("DLA width in Mpc is", dla_width*pix_spacing)
ndla = int(Npmasked_total // dla_width)
nmasked_dla = 0
dla_mask_small = np.ones((nside, nside, Np)) # store the mask
# get all indices
dla_locs_xx, dla_locs_yy = np.indices((nside, nside))
if ndla!=nside**2:
    dla_locs_xx = rng.choice(dla_locs_xx.ravel(), size=ndla, replace=False)
    dla_locs_yy = rng.choice(dla_locs_yy.ravel(), size=ndla, replace=False)
else:
    dla_locs_xx = dla_locs_xx.ravel()
    dla_locs_yy = dla_locs_yy.ravel()
dla_locs_zstart = rng.choice(np.arange(Np-dla_width), size=ndla, replace=True)
for i in range(ndla):
    z_start = dla_locs_zstart[i]
    z_end  = z_start + dla_width
    nmasked_dla += dla_width
    dla_mask_small[dla_locs_xx[i], dla_locs_yy[i], z_start:z_end] = 0
print("total number of pixels masked in DLA scheme is", nmasked_dla/Npmasked_total*100, "percent of the total desired")


#4: double skyline-like masking: mask 1 pixel 1/x of the way through the box and another pixel 1/x+2 of the way through the box
print("Doing double-skyline masking")
double_skyline_width = int(skyline_width//2)
print("double skyline width", double_skyline_width)
double_skyline_distance = 125
double_sky_maskrange = np.arange(int(Np//boxdiv), int(Np//boxdiv)+double_skyline_width)
nmasked_2skyline = double_skyline_width*2*nskew
print("total number of pixels masked in double skyline scheme is", nmasked_2skyline/Npmasked_total*100, "percent of the total desired")

skyline_2mask = np.ones((nside, nside, Np)) # store the mask
skyline_2mask[:, :, double_sky_maskrange] = 0
skyline_2mask[:, :, double_sky_maskrange+double_skyline_distance] = 0

Doing random masking
Total number of pixels masked in random scheme is 100.06869280779804 percent of the total desired
Doing single-skyline masking
total number of pixels masked in skyline scheme is 100.0 percent of the total desired
Doing DLA masking
DLA width in Mpc is 20.0
total number of pixels masked in DLA scheme is 99.99819155092592 percent of the total desired
Doing double-skyline masking
double skyline width 15


In [6]:
# reshape the masks
skyline_mask = skyline_mask.reshape((nskew, Np))
skyline_2mask = skyline_2mask.reshape((nskew, Np))
random_mask = random_mask.reshape((nskew, Np))
dla_mask = dla_mask.reshape((nskew, Np))
dla_mask_small = dla_mask_small.reshape((nskew, Np))

In [5]:
maskpath = "/data/desi/scratch/mlokken/masking/"
# save the masks
# np.save(maskpath+'skyline_mask', skyline_mask)
# np.save(maskpath+'double_skyline_mask', skyline_2mask)
# np.save(maskpath+'random_mask', random_mask)
# np.save(maskpath+'dla_mask', dla_mask)
# np.save(maskpath+'dla_mask_small', dla_mask)
np.save(maskpath+'nonmask', nonmask)